# Route Optimization Engine
### Oleh Muhammad Rosyid Suseno

# 1. Distance Matrix

In [23]:
import contextily as ctx
import folium
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import geopy.distance
from folium import plugins
from operator import itemgetter
from scipy.spatial import distance
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import pandas as pd

def load_data(file_path):
    return pd.read_csv(file_path)

def group_and_calculate_median(data):
    district_tempat = data.groupby('place_id')[['latitude', 'longitude']].median().reset_index()
    district_tempat.columns = ['place_id', 'latitude', 'longitude']
    return district_tempat

def get_highway_graph(center_location, dist):
    G = ox.graph_from_point(center_location, dist=dist, network_type='drive')
    return ox.utils_graph.get_largest_component(G, strongly=True)

def get_depot_and_nearest_nodes(G, center_location, district_tempat):
    depot = ox.distance.nearest_nodes(G, center_location[1], center_location[0])
    rute_stops = [(row['latitude'], row['longitude']) for _, row in district_tempat.iterrows()]
    node_stop = [ox.distance.nearest_nodes(G, stop[1], stop[0]) for stop in rute_stops]
    return depot, node_stop

def add_bus_stops_to_graph(G, district_tempat):
    for _, rute_stop in district_tempat.iterrows():
        nearest_node = ox.distance.nearest_nodes(G, rute_stop['longitude'], rute_stop['latitude'])
        dist = geopy.distance.distance((G.nodes[nearest_node]['y'], G.nodes[nearest_node]['x']), (rute_stop['latitude'], rute_stop['longitude']))
        G.add_node(rute_stop['place_id'], x=rute_stop['longitude'], y=rute_stop['latitude'])
        G.add_edge(nearest_node, rute_stop['place_id'], weight=dist.m)
        G.add_edge(rute_stop['place_id'], nearest_node, weight=dist.m)
    return G

def create_routing_model(nodes, NUM_VEHICLES, depot):
    manager = pywrapcp.RoutingIndexManager(len(nodes), NUM_VEHICLES, nodes.index(depot))
    routing = pywrapcp.RoutingModel(manager)
    return routing, manager

def distance_callback(from_node_index, to_node_index, nodes, G, manager):
    from_node = nodes[manager.IndexToNode(from_node_index)]
    to_node = nodes[manager.IndexToNode(to_node_index)]
    return nx.shortest_path_length(G, from_node, to_node)

def add_distance_constraint(routing, transit_callback_index, NUM_VEHICLES):
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,
        3000,
        True,
        dimension_name
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

def solve_routing_problem(routing, search_parameters):
    return routing.SolveWithParameters(search_parameters)

def print_solution(routing, solution, NUM_VEHICLES, manager, nodes, district_tempat):
    node_to_place_id = {0: 'depot'}
    for i, node in enumerate(nodes[1:], start=1):
        node_to_place_id[node] = district_tempat.iloc[i-1]['place_id']
    
    total_distance = 0
    for vehicle_id in range(NUM_VEHICLES):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route = ['depot']  # Start with 'depot'
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            if route[-1] != 'depot':  # Avoid adding depot again after the first node
                route.append(node_to_place_id[nodes[node_index]])
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        route.append('depot')  # End with 'depot'
        print(f"Route for vehicle {vehicle_id}:\n{' -> '.join(str(node) for node in route)}")
        print(f"Distance of route: {route_distance/10} km\n")
        total_distance += route_distance
    print(f"Total distance of all routes: {total_distance/10} km")

def create_map_and_plot_routes(G, depot, rute_stops_df, nodes, solution, routing, manager, NUM_VEHICLES, center_location):
    m = folium.Map(location=center_location, zoom_start=16)
    depot_coords = (G.nodes[depot]['y'], G.nodes[depot]['x'])
    folium.Marker(location=depot_coords, icon=folium.Icon(color='red', icon='home', prefix='fa'), tooltip=f"Depot {depot_coords}").add_to(m)
    for index, rute_stop in rute_stops_df.iterrows():
        stop_coords = (rute_stop['latitude'], rute_stop['longitude'])
        folium.Marker(location=stop_coords, icon=folium.Icon(color='green', icon='circle', prefix='fa'), tooltip=f"Pemberhentian {stop_coords}").add_to(m)
    colors = ['blue', 'orange', 'yellow', 'green']
    for vehicle_id in range(NUM_VEHICLES):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(nodes[node_index])
            index = solution.Value(routing.NextVar(index))
        route.append(nodes[manager.IndexToNode(index)])
        color = colors[vehicle_id % NUM_VEHICLES]
        segments = []
        for i in range(len(route)-1):
            path = nx.shortest_path(G, route[i], route[i + 1], weight='length')
            segments.append([(G.nodes[node]['y'], G.nodes[node]['x']) for node in path])
        for segment in segments:
            vehicle_tooltip = vehicle_id
            folium.PolyLine(locations=segment, color=color, weight=5, tooltip=f"Vehicle{vehicle_tooltip}").add_to(m)
            ant_path = plugins.AntPath(
                locations=segment,
                color=color,
                dash_array=[10, 50],
                delay=500,
                weight=5,
            )
            m.add_child(ant_path)
    return m

def optimize_delivery_routes(file_path, center_location, dist, NUM_VEHICLES):
    # Load data
    data = load_data(file_path)
    # Group and calculate median
    district_tempat = group_and_calculate_median(data)
    # Get the highway graph
    G = get_highway_graph(center_location, dist)
    # Get depot and nearest nodes
    depot, node_stop = get_depot_and_nearest_nodes(G, center_location, district_tempat)
    # Add bus stops to the highway graph
    G = add_bus_stops_to_graph(G, district_tempat)
    # Create routing model
    nodes = [depot] + node_stop
    routing, manager = create_routing_model(nodes, NUM_VEHICLES, depot)
    # Define distance callback
    transit_callback_index = routing.RegisterTransitCallback(lambda from_node, to_node: distance_callback(from_node, to_node, nodes, G, manager))
    # Add Distance constraint
    add_distance_constraint(routing, transit_callback_index, NUM_VEHICLES)
    # Set path-cheapest-arc search strategy
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    # Solve the problem
    solution = solve_routing_problem(routing, search_parameters)
    # Print the solution
    print_solution(routing, solution, NUM_VEHICLES, manager, nodes, district_tempat)
    # Create map and plot routes
    m = create_map_and_plot_routes(G, depot, district_tempat, nodes, solution, routing, manager, NUM_VEHICLES, center_location)
    # Display map
    return m


In [24]:
# Example usage
file_path = "data\input\Data_Alfamart Indomaret_South Jakarta_Filtered.csv"
center_location = (-6.255072, 106.797413)
dist = 10000
NUM_VEHICLES = 4
optimized_delivery_map = optimize_delivery_routes(file_path, center_location, dist, NUM_VEHICLES)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\muham\AppData\Local\Temp\ipykernel_16116\584406428.py:2: SyntaxWarning: invalid escape sequence '\i'
  file_path = "data\input\Data_Alfamart Indomaret_South Jakarta_Filtered.csv"


Route for vehicle 0:
depot -> ChIJe35tSOjwaS4R6QeisbhjlkM -> ChIJz6qqqsnwaS4R5k7p_0d1KX8 -> ChIJLWcbOM_xaS4RIMEFJuZjRvY -> ChIJd-fx6q3xaS4RLtxCS_G0RvY -> ChIJCZIzrZ_xaS4RiJX_wvOeaH4 -> depot
Distance of route: 18.8 km

Route for vehicle 1:
depot -> ChIJwX02iFLxaS4RMQgSCQVBeVQ -> ChIJPz4W5BPvaS4RoUnKBAiaL3o -> ChIJaxj8TyDuaS4RmJ8rDkpB2GA -> ChIJsVZkdYnxaS4RtDwdFb5B2t0 -> ChIJhxAVNdLzaS4RE36ihgryocM -> ChIJN7p70X7xaS4R6VpOujMI_Wo -> depot
Distance of route: 19.5 km

Route for vehicle 2:
depot -> ChIJ4T8PLOPzaS4RQDVcYaC-IGg -> ChIJpbNXVUrzaS4Rt_lHfKPzCKM -> ChIJtaqqauvzaS4RnSCep_ghRMM -> ChIJTYdPFlzzaS4R4JoaMHSAiPY -> ChIJSXpPIvDzaS4RKSuxd1gXm2Q -> depot
Distance of route: 20.1 km

Route for vehicle 3:
depot -> ChIJcQAAANntaS4RAesd4Pw1hXk -> ChIJWdlCSXPuaS4RzpIWwflyKKU -> ChIJ-6Meqf_xaS4R99u9vNNVFjg -> ChIJvy_4mvnxaS4RjjbhIFaTp4Y -> depot
Distance of route: 17.8 km

Total distance of all routes: 76.2 km


In [25]:
optimized_delivery_map

# 2. Capacitated Vehicle Routing Problem

In [16]:
import contextily as ctx
import folium
from folium import plugins
import matplotlib.pyplot as plt
import geopy.distance
import networkx as nx
import osmnx as ox
from operator import itemgetter
from scipy.spatial import distance
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import pandas as pd

def load_data(file_path):
    return pd.read_csv(file_path)

def group_and_calculate_median(data):
    district_tempat = data.groupby('place_id')[['latitude', 'longitude']].median().reset_index()
    district_tempat.columns = ['place_id', 'latitude', 'longitude']
    return district_tempat

def get_highway_graph(center_location, dist):
    G = ox.graph_from_point(center_location, dist=dist, network_type='drive')
    return ox.utils_graph.get_largest_component(G, strongly=True)

def get_depot_and_nearest_nodes(G, center_location, district_tempat):
    depot = ox.distance.nearest_nodes(G, center_location[1], center_location[0])
    rute_stops = [(row['latitude'], row['longitude']) for _, row in district_tempat.iterrows()]
    node_stop = [ox.distance.nearest_nodes(G, stop[1], stop[0]) for stop in rute_stops]
    return depot, node_stop

def add_bus_stops_to_graph(G, district_tempat):
    for _, rute_stop in district_tempat.iterrows():
        nearest_node = ox.distance.nearest_nodes(G, rute_stop['longitude'], rute_stop['latitude'])
        dist = geopy.distance.distance((G.nodes[nearest_node]['y'], G.nodes[nearest_node]['x']), (rute_stop['latitude'], rute_stop['longitude']))
        G.add_node(rute_stop['place_id'], x=rute_stop['longitude'], y=rute_stop['latitude'])
        G.add_edge(nearest_node, rute_stop['place_id'], weight=dist.m)
        G.add_edge(rute_stop['place_id'], nearest_node, weight=dist.m)
    return G

def create_routing_model(nodes, NUM_VEHICLES, depot, demands, vehicle_capacities):
    manager = pywrapcp.RoutingIndexManager(len(nodes), NUM_VEHICLES, nodes.index(depot))
    routing = pywrapcp.RoutingModel(manager)

    return routing, manager

def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return demands[from_node]

def distance_callback(from_node_index, to_node_index, nodes, G, manager):
    from_node = nodes[manager.IndexToNode(from_node_index)]
    to_node = nodes[manager.IndexToNode(to_node_index)]
    return nx.shortest_path_length(G, from_node, to_node)

def add_distance_constraint(routing, transit_callback_index, NUM_VEHICLES):
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,
        3000,
        True,
        dimension_name
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

def solve_routing_problem(routing, search_parameters):
    return routing.SolveWithParameters(search_parameters)

def print_solution(routing, solution, NUM_VEHICLES, manager, vehicle_capacities):
    total_distance = 0
    total_load = 0
    for vehicle_id in range(NUM_VEHICLES):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
            route_load += demands[node_index]
        route.append(manager.IndexToNode(index))
        print(f"Route for vehicle {vehicle_id}:")
        print(f"{' -> '.join(str(node) + ' Load(' + str(demands[node]) + ')' for node in route)}")
        print(f"Distance of route: {route_distance/10} km")
        print(f"Load of route: {route_load}\n")
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance/10} km")
    print(f"Total load of all routes: {total_load}")

def create_map_and_plot_routes(G, depot, rute_stops_df, nodes, solution, routing, manager, NUM_VEHICLES, center_location):
    m = folium.Map(location=center_location, zoom_start=16)
    depot_coords = (G.nodes[depot]['y'], G.nodes[depot]['x'])
    folium.Marker(location=depot_coords, icon=folium.Icon(color='red', icon='home', prefix='fa'), tooltip=f"Depot {depot_coords}").add_to(m)
    for index, rute_stop in rute_stops_df.iterrows():
        stop_coords = (rute_stop['latitude'], rute_stop['longitude'])
        folium.Marker(location=stop_coords, icon=folium.Icon(color='green', icon='circle', prefix='fa'), tooltip=f"Pemberhentian {stop_coords}").add_to(m)
    colors = ['blue', 'orange', 'yellow', 'green']
    for vehicle_id in range(NUM_VEHICLES):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(nodes[node_index])
            index = solution.Value(routing.NextVar(index))
        route.append(nodes[manager.IndexToNode(index)])
        color = colors[vehicle_id % NUM_VEHICLES]
        segments = []
        for i in range(len(route)-1):
            path = nx.shortest_path(G, route[i], route[i + 1], weight='length')
            segments.append([(G.nodes[node]['y'], G.nodes[node]['x']) for node in path])
        for segment in segments:
            vehicle_tooltip = vehicle_id
            folium.PolyLine(locations=segment, color=color, weight=5, tooltip=f"Vehicle{vehicle_tooltip}").add_to(m)
            ant_path = plugins.AntPath(
                locations=segment,
                color=color,
                dash_array=[10, 50],
                delay=500,
                weight=5,
            )
            m.add_child(ant_path)
    return m

def optimize_delivery_routes(file_path, center_location, dist, NUM_VEHICLES, vehicle_capacities, demands):
    # Load data
    data = load_data(file_path)
    district_tempat = group_and_calculate_median(data)
    # Get the highway graph
    G = get_highway_graph(center_location, dist)
    # Get depot and nearest nodes
    depot, node_stop = get_depot_and_nearest_nodes(G, center_location, district_tempat)
    
    G = add_bus_stops_to_graph(G, district_tempat)
    # Create routing model
    nodes = [depot] + node_stop
    routing, manager = create_routing_model(nodes, NUM_VEHICLES, depot, demands, vehicle_capacities)
    # Define distance callback
    transit_callback_index = routing.RegisterTransitCallback(lambda from_node, to_node: distance_callback(from_node, to_node, nodes, G, manager))
    # Add Distance constraint
    add_distance_constraint(routing, transit_callback_index, NUM_VEHICLES)
    # Set path-cheapest-arc search strategy
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    # Solve the problem
    solution = solve_routing_problem(routing, search_parameters)
    # Print the solution
    print_solution(routing, solution, NUM_VEHICLES, manager, vehicle_capacities)
    # Create map and plot routes
    m = create_map_and_plot_routes(G, depot, district_tempat, nodes, solution, routing, manager, NUM_VEHICLES, center_location)
    # Display map
    return m

demands = [0, 1, 1, 1, 2, 2, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8, 8]
vehicle_capacities = [20, 20, 20, 20]

In [17]:

# Aturlah parameter lainnya
file_path = "data\input\Data_Alfamart Indomaret_South Jakarta_Filtered.csv"
center_location = (-6.255072, 106.797413)
dist = 10000  # Jarak dalam meter
NUM_VEHICLES = 4  # Jumlah kendaraan

# Call the optimization function
optimize_delivery_routes(file_path, center_location, dist, NUM_VEHICLES, vehicle_capacities, demands)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\muham\AppData\Local\Temp\ipykernel_7616\374057215.py:2: SyntaxWarning: invalid escape sequence '\i'
  file_path = "data\input\Data_Alfamart Indomaret_South Jakarta_Filtered.csv"


Route for vehicle 0:
0 Load(0) -> 13 Load(2) -> 20 Load(8) -> 4 Load(2) -> 12 Load(1) -> 3 Load(1) -> 0 Load(0)
Distance of route: 188m
Load of route: 14

Route for vehicle 1:
0 Load(0) -> 19 Load(8) -> 6 Load(2) -> 10 Load(8) -> 16 Load(4) -> 14 Load(1) -> 5 Load(2) -> 0 Load(0)
Distance of route: 195m
Load of route: 25

Route for vehicle 2:
0 Load(0) -> 2 Load(1) -> 15 Load(2) -> 17 Load(4) -> 8 Load(2) -> 7 Load(4) -> 0 Load(0)
Distance of route: 201m
Load of route: 13

Route for vehicle 3:
0 Load(0) -> 11 Load(8) -> 9 Load(4) -> 1 Load(1) -> 18 Load(8) -> 0 Load(0)
Distance of route: 178m
Load of route: 21

Total distance of all routes: 762m
Total load of all routes: 73


---

# 3. Capacitated Vehicle Routing Problem with Time Windows

---

Route for vehicle 0:
0 Load(0) -> 13 Load(2) -> 20 Load(8) -> 4 Load(2) -> 12 Load(1) -> 3 Load(1) -> 0 Load(0)
Distance of route: 188m
Load of route: 14

Route for vehicle 1:
0 Load(0) -> 19 Load(8) -> 6 Load(2) -> 10 Load(8) -> 16 Load(4) -> 14 Load(1) -> 5 Load(2) -> 0 Load(0)
Distance of route: 195m
Load of route: 25

Route for vehicle 2:
0 Load(0) -> 2 Load(1) -> 15 Load(2) -> 17 Load(4) -> 8 Load(2) -> 7 Load(4) -> 0 Load(0)
Distance of route: 201m
Load of route: 13

Route for vehicle 3:
0 Load(0) -> 11 Load(8) -> 9 Load(4) -> 1 Load(1) -> 18 Load(8) -> 0 Load(0)
Distance of route: 178m
Load of route: 21

Total distance of all routes: 762m
Total load of all routes: 73

In [4]:
data = 'data/input/Data_Alfamart Indomaret_South Jakarta.csv'
dt = pd.read_csv(data)
dt

,nama_tempat,rating_tempat,user_ratings_total,latitude,longitude,alamat_tempat,place_id,store,place_id.1,nama_kelurahan,nama_kecamatan,nama_kota
0,indomaret,4.3,104,-6.302203,106.791936,"jl. lb. bulus iii no.40, rt.9/rw.7, cilandak b...",ChIJaxj8TyDuaS4RmJ8rDkpB2GA,Indomaret,ChIJaxj8TyDuaS4RmJ8rDkpB2GA,Cilandak Barat,Cilandak,Kota Jakarta Selatan
1,indomaret,0.0,0,-6.307003,106.793690,"10, rt.4/rw.10, pondok labu, south jakarta city",ChIJPz4W5BPvaS4RoUnKBAiaL3o,Indomaret,ChIJPz4W5BPvaS4RoUnKBAiaL3o,Pondok Labu,Cilandak,Kota Jakarta Selatan
2,indomaret bdn raya,4.2,66,-6.279392,106.798442,"jl. bdn raya no.10, rt.10/rw.11, cilandak bar....",ChIJKZqyypTxaS4RkjwUtyXrkBY,Indomaret,ChIJKZqyypTxaS4RkjwUtyXrkBY,Cilandak Barat,Cilandak,Kota Jakarta Selatan
3,indomaret,4.7,3,-6.278223,106.797096,"jl. rs. fatmawati raya no.7, rt.8/rw.6, gandar...",ChIJzVqDlcTxaS4Rwlnqrz5hXbo,Indomaret,ChIJzVqDlcTxaS4Rwlnqrz5hXbo,Gandaria Selatan,Cilandak,Kota Jakarta Selatan
4,indomaret karang tengah raya,4.3,108,-6.301344,106.780550,"bona indah plaza, jl. karang tengah raya no.1,...",ChIJ7aqqqinuaS4RKy68g0RuyMk,Indomaret,ChIJ7aqqqinuaS4RKy68g0RuyMk,Lebak Bulus,Cilandak,Kota Jakarta Selatan
...,...,...,...,...,...,...,...,...,...,...,...,...
670,alfamart komplek selmis,0.0,0,-6.225834,106.858991,"jl. asem baris raya no.3, rt.3/rw.9, kebon bar...",ChIJ0dPodPrzaS4RE47Rr8Mz77Y,Alfamart,ChIJ0dPodPrzaS4RE47Rr8Mz77Y,Kebon Baru,Tebet,Kota Jakarta Selatan
671,alfamart bukit duri selatan,0.0,0,-6.221214,106.856739,"jl. cucakrawa no.882, rt.8/rw.4, bukit duri, k...",ChIJ8a2fcwfzaS4Rte3SBP5Gj_0,Alfamart,ChIJ8a2fcwfzaS4Rte3SBP5Gj_0,Bukit Duri,Tebet,Kota Jakarta Selatan
672,alfamart stasiu,5.0,3,-6.225869,106.858647,"qvf5+mfc, flyover stasiun tebet, rt.3/rw.9, kb...",ChIJYRBiI-rzaS4RBW5LKzm2-yY,Alfamart,ChIJYRBiI-rzaS4RBW5LKzm2-yY,Kebon Baru,Tebet,Kota Jakarta Selatan
673,alfamart barkah ii,0.0,0,-6.219769,106.851183,"jl. barkah i no.14, rw.6, manggarai sel., kota...",ChIJ85lVEnTzaS4RHy8f_3cmsuI,Alfamart,ChIJ85lVEnTzaS4RHy8f_3cmsuI,Manggarai Selatan,Tebet,Kota Jakarta Selatan


In [6]:
# Drop specified columns
columns_to_drop = ['nama_kelurahan', 'place_id.1', 'rating_tempat', 'user_ratings_total']
dt_dropped = dt.drop(columns=columns_to_drop)
dt_dropped

,nama_tempat,latitude,longitude,alamat_tempat,place_id,store,nama_kecamatan,nama_kota
0,indomaret,-6.302203,106.791936,"jl. lb. bulus iii no.40, rt.9/rw.7, cilandak b...",ChIJaxj8TyDuaS4RmJ8rDkpB2GA,Indomaret,Cilandak,Kota Jakarta Selatan
1,indomaret,-6.307003,106.793690,"10, rt.4/rw.10, pondok labu, south jakarta city",ChIJPz4W5BPvaS4RoUnKBAiaL3o,Indomaret,Cilandak,Kota Jakarta Selatan
2,indomaret bdn raya,-6.279392,106.798442,"jl. bdn raya no.10, rt.10/rw.11, cilandak bar....",ChIJKZqyypTxaS4RkjwUtyXrkBY,Indomaret,Cilandak,Kota Jakarta Selatan
3,indomaret,-6.278223,106.797096,"jl. rs. fatmawati raya no.7, rt.8/rw.6, gandar...",ChIJzVqDlcTxaS4Rwlnqrz5hXbo,Indomaret,Cilandak,Kota Jakarta Selatan
4,indomaret karang tengah raya,-6.301344,106.780550,"bona indah plaza, jl. karang tengah raya no.1,...",ChIJ7aqqqinuaS4RKy68g0RuyMk,Indomaret,Cilandak,Kota Jakarta Selatan
...,...,...,...,...,...,...,...,...
670,alfamart komplek selmis,-6.225834,106.858991,"jl. asem baris raya no.3, rt.3/rw.9, kebon bar...",ChIJ0dPodPrzaS4RE47Rr8Mz77Y,Alfamart,Tebet,Kota Jakarta Selatan
671,alfamart bukit duri selatan,-6.221214,106.856739,"jl. cucakrawa no.882, rt.8/rw.4, bukit duri, k...",ChIJ8a2fcwfzaS4Rte3SBP5Gj_0,Alfamart,Tebet,Kota Jakarta Selatan
672,alfamart stasiu,-6.225869,106.858647,"qvf5+mfc, flyover stasiun tebet, rt.3/rw.9, kb...",ChIJYRBiI-rzaS4RBW5LKzm2-yY,Alfamart,Tebet,Kota Jakarta Selatan
673,alfamart barkah ii,-6.219769,106.851183,"jl. barkah i no.14, rw.6, manggarai sel., kota...",ChIJ85lVEnTzaS4RHy8f_3cmsuI,Alfamart,Tebet,Kota Jakarta Selatan


In [20]:
kecamatan_unique = dt_dropped['nama_kecamatan'].unique()
kecamatan_unique

array(['Cilandak', 'Jagakarsa', 'Kebayoran Baru', 'Kebayoran Lama',
       'Mampang Prapatan', 'Pancoran', 'Pasar Minggu', 'Pesanggrahan',
       'Setiabudi', 'Tebet'], dtype=object)

In [21]:
import pandas as pd
dataset = 'data/input/Data_Alfamart Indomaret_South Jakarta_Filtered.csv'
df = pd.read_csv(dataset)

In [24]:
# Ganti 'indomaret' dengan 'alfamart' di kolom 'store' dan 'nama_tempat'
df['store'] = df['store'].str.replace('indomaret', 'alfamart', case=False)
df['nama_tempat'] = df['nama_tempat'].str.replace('indomaret', 'alfamart', case=False)
columns_to_drop = ['nama_kelurahan', 'place_id.1', 'rating_tempat', 'user_ratings_total']
df = df.drop(columns=columns_to_drop)

In [26]:
df

,nama_tempat,latitude,longitude,alamat_tempat,place_id,store,nama_kecamatan,nama_kota
0,alfamart,-6.302203,106.791936,"jl. lb. bulus iii no.40, rt.9/rw.7, cilandak b...",ChIJaxj8TyDuaS4RmJ8rDkpB2GA,alfamart,Cilandak,Kota Jakarta Selatan
1,alfamart,-6.307003,106.793690,"10, rt.4/rw.10, pondok labu, south jakarta city",ChIJPz4W5BPvaS4RoUnKBAiaL3o,alfamart,Cilandak,Kota Jakarta Selatan
2,alfamart,-6.320362,106.810676,"jl. m. khafi no.62, rt.3/rw.4, jagakarsa, kota...",ChIJWdlCSXPuaS4RzpIWwflyKKU,alfamart,Jagakarsa,Kota Jakarta Selatan
3,alfamart ciremai jagakarsa,-6.324509,106.811308,"jl. m.kahfi 1 rt 01/rw 04, jagakarsa, jagakars...",ChIJcQAAANntaS4RAesd4Pw1hXk,alfamart,Jagakarsa,Kota Jakarta Selatan
4,alfamart,-6.255072,106.797413,"jl. darmawangsa raya no.125, rt.2, rt.2/rw.1, ...",ChIJwX02iFLxaS4RMQgSCQVBeVQ,alfamart,Kebayoran Baru,Kota Jakarta Selatan
5,alfamart panglima polim - blok a,-6.257676,106.796800,"jl. rs. fatmawati raya no.1, gandaria utara, k...",ChIJCZIzrZ_xaS4RiJX_wvOeaH4,alfamart,Kebayoran Baru,Kota Jakarta Selatan
6,alfamart,-6.285777,106.780884,"pq7j+p92, jl. metro pondok indah, rt.3/rw.14, ...",ChIJLWcbOM_xaS4RIMEFJuZjRvY,alfamart,Kebayoran Lama,Kota Jakarta Selatan
7,alfamart ciputat raya 4,-6.282964,106.780607,"jl. ciputat raya no.4, rt.9/rw.2, pd. pinang, ...",ChIJd-fx6q3xaS4RLtxCS_G0RvY,alfamart,Kebayoran Lama,Kota Jakarta Selatan
8,fresh alfamart kemang,-6.272598,106.814950,"jl. kemang raya no.128, rt.3/rw.2, bangka, kot...",ChIJsVZkdYnxaS4RtDwdFb5B2t0,alfamart,Mampang Prapatan,Kota Jakarta Selatan
9,alfamart kemang mampang tjkc,-6.255553,106.812338,"jl. kemang raya no.15, rt.13/rw.1, bangka, kot...",ChIJN7p70X7xaS4R6VpOujMI_Wo,alfamart,Mampang Prapatan,Kota Jakarta Selatan


In [27]:
import folium
def create_map_and_plot_routes(depot, rute_stops_df, center_location):
    m = folium.Map(location=center_location, zoom_start=10)
    folium.Marker(location=depot, icon=folium.Icon(color='red', icon='home', prefix='fa'), tooltip="Depot").add_to(m)
    for index, rute_stop in rute_stops_df.iterrows():
        stop_coords = (rute_stop['latitude'], rute_stop['longitude'])
        folium.Marker(location=stop_coords, icon=folium.Icon(color='green', icon='circle', prefix='fa'), tooltip="Pemberhentian").add_to(m)
    return m

depot_location = (-6.255072, 106.797413)
center_location = depot_location

# Buat plot peta OpenStreetMap
customer_map = create_map_and_plot_routes(depot_location, df, center_location)
customer_map

In [2]:
import contextily as ctx
import folium
from folium import plugins
import matplotlib.pyplot as plt
import geopy.distance
import networkx as nx
import osmnx as ox
import os
from operator import itemgetter
from scipy.spatial import distance
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import pandas as pd

def load_data(file_path):
    return pd.read_csv(file_path)

def group_and_calculate_median(data):
    district_tempat = data.groupby('place_id')[['latitude', 'longitude']].median().reset_index()
    district_tempat.columns = ['place_id', 'latitude', 'longitude']
    return district_tempat

def get_highway_graph(center_location, dist):
    G = ox.graph_from_point(center_location, dist=dist, network_type='drive')
    return ox.utils_graph.get_largest_component(G, strongly=True)

def get_depot_and_nearest_nodes(G, center_location, district_tempat):
    depot = ox.distance.nearest_nodes(G, center_location[1], center_location[0])
    rute_stops = [(row['latitude'], row['longitude']) for _, row in district_tempat.iterrows()]
    node_stop = [ox.distance.nearest_nodes(G, stop[1], stop[0]) for stop in rute_stops]
    return depot, node_stop

def add_bus_stops_to_graph(G, district_tempat):
    for _, rute_stop in district_tempat.iterrows():
        nearest_node = ox.distance.nearest_nodes(G, rute_stop['longitude'], rute_stop['latitude'])
        dist = geopy.distance.distance((G.nodes[nearest_node]['y'], G.nodes[nearest_node]['x']), (rute_stop['latitude'], rute_stop['longitude']))
        G.add_node(rute_stop['place_id'], x=rute_stop['longitude'], y=rute_stop['latitude'])
        G.add_edge(nearest_node, rute_stop['place_id'], weight=dist.m)
        G.add_edge(rute_stop['place_id'], nearest_node, weight=dist.m)
    return G

def create_routing_model(nodes, NUM_VEHICLES, depot, demands, vehicle_capacities):
    manager = pywrapcp.RoutingIndexManager(len(nodes), NUM_VEHICLES, nodes.index(depot))
    routing = pywrapcp.RoutingModel(manager)
    # Add capacity dimension
    demand_callback_index = routing.RegisterUnaryTransitCallback(lambda from_index: demands[manager.IndexToNode(from_index)])
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        vehicle_capacities,  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity'
    )    
    return routing, manager

def time_callback(from_node_index, to_node_index, manager):
  from_node = manager.IndexToNode(from_node_index)
  to_node = manager.IndexToNode(to_node_index)
  return time_windows[to_node][0]  # Return the start of the time window for the 'to_node'

def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return demands[from_node]

def distance_callback(from_node_index, to_node_index, nodes, G, manager):
    from_node = nodes[manager.IndexToNode(from_node_index)]
    to_node = nodes[manager.IndexToNode(to_node_index)]
    return nx.shortest_path_length(G, from_node, to_node)

def add_time_windows(routing, manager, time_windows, transit_callback_index):
    time = 'Time'
    max_time = max(window[1] for window in time_windows)  # Mengambil nilai maksimum dari time windows
    routing.AddDimension(
        transit_callback_index,
        30,  # waiting time
        max_time,  # maximum time per node
        False,  # start cumul to zero
        time
    )
    time_dimension = routing.GetDimensionOrDie(time)
    for node_index, window in enumerate(time_windows):
        time_dimension.CumulVar(manager.NodeToIndex(node_index)).SetRange(window[0], window[1])

def add_distance_constraint(routing, transit_callback_index, NUM_VEHICLES):
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,
        3000,
        True,
        dimension_name
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

def add_distance_and_time_constraints(routing, manager, demand_callback_index, vehicle_capacities_list, time_windows, transit_callback_index, time_callback_index, NUM_VEHICLES):
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    routing.AddDimension(transit_callback_index, 0, 3000, True, 'Distance')
    distance_dimension = routing.GetDimensionOrDie('Distance')
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Add capacity dimension for each vehicle separately
    for i in range(NUM_VEHICLES):
        routing.AddDimensionWithVehicleCapacity(
            demand_callback_index, 0, vehicle_capacities_list[i], True, f'Capacity_{i}'
        )

    routing.AddDimension(
        time_callback_index,
        30,  # waiting time
        1440,  # maximum time in a day
        True,
        'Time'
    )
    time_dimension = routing.GetDimensionOrDie('Time')
    time_dimension.SetGlobalSpanCostCoefficient(100)


def solve_routing_problem(routing, search_parameters):
    return routing.SolveWithParameters(search_parameters)

def print_solution(routing, solution, NUM_VEHICLES, manager, vehicle_capacities, time_windows, nodes, G, depot):
    total_distance = 0
    total_load = 0
    total_time = 0
    for vehicle_id in range(NUM_VEHICLES):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0
        route_time = 0
        route = []
        route.append(depot)  # Mulai dari depot
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            # Menghitung jarak menggunakan distance_callback
            route_distance += distance_callback(previous_index, index, nodes, G, manager)
            # Menghitung waktu menggunakan time_callback
            route_time += time_callback(previous_index, index, manager)
            route_load += demands[node_index]
        route.append(depot)  # Selesai di depot
        print(f"Route for vehicle {vehicle_id}:")
        route_info = []
        for node in route:
            if node == depot:
                node_info = f"[Depot: Load({demands[0]}) Time({time_windows[0][0]}, {time_windows[0][1]})]"  # Depot memiliki waktu dari time_windows[0]
            else:
                node_info = f"\n[Node {node}: Load({demands[node]}) Time({time_windows[node][0]}, {time_windows[node][1]})]"
            route_info.append(node_info)
        print(" -> ".join(route_info))
        print(f"Distance of the route: {route_distance/10} km")  # Menggunakan jarak yang dihitung selama proses routing
        print(f"Load of the route: {route_load}")
        print(f"Time of the route: {route_time/10} min\n")  # Menggunakan waktu yang dihitung selama proses routing
        total_distance += route_distance
        total_load += route_load
        total_time += route_time
    print(f"Total distance of all routes: {total_distance/10} km")
    print(f"Total load of all routes: {total_load}")
    print(f"Total time of all routes: {total_time/10}min")


def create_map_and_plot_routes(G, depot, rute_stops_df, nodes, solution, routing, manager, NUM_VEHICLES, center_location):
    m = folium.Map(location=center_location, zoom_start=16)
    depot_coords = (G.nodes[depot]['y'], G.nodes[depot]['x'])
    folium.Marker(location=depot_coords, icon=folium.Icon(color='red', icon='home', prefix='fa'), tooltip=f"Depot {depot_coords}").add_to(m)
    for index, rute_stop in rute_stops_df.iterrows():
        stop_coords = (rute_stop['latitude'], rute_stop['longitude'])
        folium.Marker(location=stop_coords, icon=folium.Icon(color='green', icon='circle', prefix='fa'), tooltip=f"Pemberhentian {stop_coords}").add_to(m)
    colors = ['blue', 'orange', 'yellow', 'green']
    for vehicle_id in range(NUM_VEHICLES):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(nodes[node_index])
            index = solution.Value(routing.NextVar(index))
        route.append(nodes[manager.IndexToNode(index)])
        color = colors[vehicle_id % NUM_VEHICLES]
        segments = []
        for i in range(len(route)-1):
            path = nx.shortest_path(G, route[i], route[i + 1], weight='length')
            segments.append([(G.nodes[node]['y'], G.nodes[node]['x']) for node in path])
        for segment in segments:
            vehicle_tooltip = vehicle_id
            folium.PolyLine(locations=segment, color=color, weight=5, tooltip=f"Vehicle{vehicle_tooltip}").add_to(m)
            ant_path = plugins.AntPath(
                locations=segment,
                color=color,
                dash_array=[10, 50],
                delay=500,
                weight=5,
            )
            m.add_child(ant_path)
    return m


def optimize_delivery_routes(file_path, center_location, dist, NUM_VEHICLES, vehicle_capacities, demands, time_windows):
    # Load data
    data = load_data(file_path)
    district_tempat = group_and_calculate_median(data)
    # Get the highway graph
    G = get_highway_graph(center_location, dist)
    # Get depot and nearest nodes
    depot, node_stop = get_depot_and_nearest_nodes(G, center_location, district_tempat)
    
    G = add_bus_stops_to_graph(G, district_tempat)
    # Create routing model
    nodes = [depot] + node_stop
    routing, manager = create_routing_model(nodes, NUM_VEHICLES, depot, demands, vehicle_capacities)
    # Define distance callback
    transit_callback_index = routing.RegisterTransitCallback(lambda from_node, to_node: distance_callback(from_node, to_node, nodes, G, manager))
    # Add Distance constraint
    add_distance_constraint(routing, transit_callback_index, NUM_VEHICLES)
    # Set path-cheapest-arc search strategy
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    # Solve the problem
    solution = solve_routing_problem(routing, search_parameters)
    # Print the solution
    print_solution(routing, solution, NUM_VEHICLES, manager, vehicle_capacities, time_windows, nodes, G, depot)
    # Create map and plot routes
    m = create_map_and_plot_routes(G, depot, district_tempat, nodes, solution, routing, manager, NUM_VEHICLES, center_location)
    # Display map
    return m

file_path = os.path.join("data", "input", "Data_Alfamart Indomaret_South Jakarta_Filtered.csv")
center_location = (-6.255072, 106.797413)
dist = 10000  # Jarak dalam meter
NUM_VEHICLES = 4  # Jumlah kendaraan
demands = [0, 1, 1, 1, 2, 2, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8, 8]
vehicle_capacities = [20, 20, 20, 20]
time_windows= [
 (0,  160),
 (80,  115), #1
 (25,  55), #2
 (75,  100), #3
 (80,  110), #4
 (120,  150), #5
 (105,  125), #6
 (40,  65), #7
 (10,  25), #8
 (45,  85), #9
 (105,  130), #10
 (10,  40), #11
 (50,  85), #12
 (105,  125), #13
 (25,  50), #14
 (15,  50), #15
 (50,  75), #16
 (45,  85), #17
 (125,  150), #18
 (90,  120), #19
 (80,  110), #20
]

In [3]:
# Panggil fungsi optimasi
optimize_delivery_routes(file_path, center_location, dist, NUM_VEHICLES, vehicle_capacities, demands, time_windows)

Route for vehicle 0:
[Depot: Load(0) Time(0, 160)] -> 
[Node 0: Load(0) Time(0, 160)] -> 
[Node 8: Load(2) Time(10, 25)] -> 
[Node 14: Load(1) Time(25, 50)] -> 
[Node 7: Load(4) Time(40, 65)] -> 
[Node 16: Load(4) Time(50, 75)] -> 
[Node 3: Load(1) Time(75, 100)] -> 
[Node 19: Load(8) Time(90, 120)] -> [Depot: Load(0) Time(0, 160)]
Distance of the route: 17.6 km
Load of the route: 20
Time of the route: 29.0 min

Route for vehicle 1:
[Depot: Load(0) Time(0, 160)] -> 
[Node 0: Load(0) Time(0, 160)] -> 
[Node 2: Load(1) Time(25, 55)] -> 
[Node 12: Load(1) Time(50, 85)] -> 
[Node 4: Load(2) Time(80, 110)] -> 
[Node 10: Load(8) Time(105, 130)] -> 
[Node 18: Load(8) Time(125, 150)] -> [Depot: Load(0) Time(0, 160)]
Distance of the route: 22.8 km
Load of the route: 20
Time of the route: 38.5 min

Route for vehicle 2:
[Depot: Load(0) Time(0, 160)] -> 
[Node 0: Load(0) Time(0, 160)] -> 
[Node 15: Load(2) Time(15, 50)] -> 
[Node 17: Load(4) Time(45, 85)] -> 
[Node 20: Load(8) Time(80, 110)] -> 
[